In [1]:
import sys
sys.path.append('../src/')

In [2]:
import os
from dotenv import load_dotenv


# Load the variables from .env file
load_dotenv()

# Define the global variables to load AWS credentials and dataset file names
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_DATA_DIRECTORY = os.environ.get("AWS_S3_DATA_DIRECTORY")
AWS_S3_CLEAN_DATA_DIRECTORY = os.environ.get("AWS_S3_CLEAN_DATA_DIRECTORY")
AWS_S3_DATA_DIRECTORY_RAW = os.environ.get("AWS_S3_DATA_DIRECTORY_RAW")
APPLICATION_DATASET = os.environ.get("APPLICATION_DATASET")
AWS_S3_DATA_DIRECTORY_PROCESSED = os.environ.get("AWS_S3_DATA_DIRECTORY_PROCESSED")
AWS_S3_DATA_DIRECTORY_MODELS = os.environ.get("AWS_S3_DATA_DIRECTORY_MODELS")
DEMO_DATASET = os.environ.get("DEMO_DATASET")

In [31]:
from utils.s3_Functions import S3Utils
# Create an instance of S3Utils class to access various methods
s3_utils = S3Utils(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_S3_BUCKET, AWS_S3_DATA_DIRECTORY)

In [4]:
applicationDf = s3_utils.load_dataframe('original', 'application_data.csv')

Loading 's3://loan-default-prediction/dataset-3/original/application_data.csv' file as dataframe.


In [5]:
previousDf = s3_utils.load_dataframe('original', 'previous_application.csv')

Loading 's3://loan-default-prediction/dataset-3/original/previous_application.csv' file as dataframe.


In [6]:
import pandas as pd
loan_process_df = pd.merge(applicationDf, previousDf, how="inner", on="SK_ID_CURR")

In [7]:
loan_process_df.shape

(1413701, 158)

In [22]:
from typing import Tuple
from sklearn.model_selection import train_test_split

def split_data(
    df: pd.DataFrame, target_column: str, test_size: float = 0.2, random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Function to split the data into train and test samples"""

    # Split the features and target variables
    X, y = df.drop(target_column, axis=1), df[target_column]

    # Split the data into training and test samples
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state,
    )

    return X, X_train, X_test, y_train, y_test

In [29]:
Train_set, Test_set = train_test_split(loan_process_df, test_size=0.075)

ValueError: Input contains NaN

In [24]:
print("Train_set:", Train_set.shape)
print("Test_set:", Test_set.shape)

Train_set: (1307673, 158)
Test_set: (106028, 158)


In [25]:
Train_set[Train_set.select_dtypes(include='object').columns].value_counts()

NAME_CONTRACT_TYPE_x  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  NAME_TYPE_SUITE_x  NAME_INCOME_TYPE      NAME_EDUCATION_TYPE            NAME_FAMILY_STATUS  NAME_HOUSING_TYPE    OCCUPATION_TYPE  WEEKDAY_APPR_PROCESS_START_x  ORGANIZATION_TYPE       FONDKAPREMONT_MODE     HOUSETYPE_MODE  WALLSMATERIAL_MODE  EMERGENCYSTATE_MODE  NAME_CONTRACT_TYPE_y  WEEKDAY_APPR_PROCESS_START_y  FLAG_LAST_APPL_PER_CONTRACT  NAME_CASH_LOAN_PURPOSE  NAME_CONTRACT_STATUS  NAME_PAYMENT_TYPE      CODE_REJECT_REASON  NAME_TYPE_SUITE_y  NAME_CLIENT_TYPE  NAME_GOODS_CATEGORY   NAME_PORTFOLIO  NAME_PRODUCT_TYPE  CHANNEL_TYPE             NAME_SELLER_INDUSTRY  NAME_YIELD_GROUP  PRODUCT_COMBINATION        
Cash loans            F            N             Y                Group of people    Commercial associate  Secondary / secondary special  Married             House / apartment    Sales staff      WEDNESDAY                     Trade: type 3           not specified          block of flats  Stone, brick        No  

In [26]:
Test_set[Test_set.select_dtypes(include='object').columns].value_counts()

NAME_CONTRACT_TYPE_x  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  NAME_TYPE_SUITE_x  NAME_INCOME_TYPE      NAME_EDUCATION_TYPE            NAME_FAMILY_STATUS    NAME_HOUSING_TYPE  OCCUPATION_TYPE        WEEKDAY_APPR_PROCESS_START_x  ORGANIZATION_TYPE       FONDKAPREMONT_MODE     HOUSETYPE_MODE  WALLSMATERIAL_MODE  EMERGENCYSTATE_MODE  NAME_CONTRACT_TYPE_y  WEEKDAY_APPR_PROCESS_START_y  FLAG_LAST_APPL_PER_CONTRACT  NAME_CASH_LOAN_PURPOSE  NAME_CONTRACT_STATUS  NAME_PAYMENT_TYPE      CODE_REJECT_REASON  NAME_TYPE_SUITE_y  NAME_CLIENT_TYPE  NAME_GOODS_CATEGORY  NAME_PORTFOLIO  NAME_PRODUCT_TYPE  CHANNEL_TYPE             NAME_SELLER_INDUSTRY  NAME_YIELD_GROUP  PRODUCT_COMBINATION        
Cash loans            F            N             N                Unaccompanied      Working               Secondary / secondary special  Single / not married  House / apartment  Sales staff            MONDAY                        Business Entity Type 3  reg oper account       block of flats  Stone, brick 

In [27]:
s3_utils.save_dataframe(
          AWS_S3_DATA_DIRECTORY_RAW, 'merged_application_data.csv', Train_set
       )

Saving dataframe as 'merged_application_data.csv' at 's3://loan-default-prediction/dataset-3/raw/merged_application_data.csv'
Error saving dataframe as csv to S3: 'S3FileSystem' object is not callable


In [28]:
s3_utils.save_dataframe(
          AWS_S3_DATA_DIRECTORY_RAW, 'merged_demo_data.csv', Test_set
       )

Saving dataframe as 'merged_demo_data.csv' at 's3://loan-default-prediction/dataset-3/raw/merged_demo_data.csv'
Error saving dataframe as csv to S3: 'S3FileSystem' object is not callable
